# Module 1, Activity 3: Connecting AWS Services to an LLM Chain

In this notebook we will see some basic pipelines connecting various AWS services to Bedrock for using in an LLM.

In [ ]:
import json
from pprint import pprint
import boto3

from langchain.chains import LLMChain
from langchain_aws import ChatBedrock
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableLambda

In [ ]:
session = boto3.session.Session()
region = session.region_name

## About this cell

This is a helper function to download data from a file in an S3 bucket.  You can also make the connection directly, if you choose.

In [ ]:
def get_data_from_s3(bucket_name, key):
    s3 = boto3.client(
        's3',
        region_name=region,
    )
    response = s3.get_object(Bucket=bucket_name, Key=key)
    data = response['Body'].read().decode('utf-8')

    return data

We now make the connection to the bucket and pull down a specific text file.

In [ ]:
s3_data = get_data_from_s3("...", "constitution.txt")
s3_data[0:200]

## The prompt

Here we are creating our system and human prompts, as before.  Notice that we have provided the system prompt with a bit of instruction on what it is expected to do, namely summarize the provided text.  This text is passed in through the `s3_data` variable in the human prompt.  Again, don't forget that data that you pass in takes up tokens and must fit within the context window.  Anything that doesn't fit will not be used.

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant that specializes in summarizing documents."
)
human_prompt = HumanMessagePromptTemplate.from_template(
    "Summarize the following document concisely:\n\n{s3_data}"
)

prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

# The chain

As before, we are creating our chain using LCEL of the prompt, the model, and the output parser.

In [ ]:
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=region,
    temperature=0.0,
    max_tokens=200,
)

chain = prompt | llm | StrOutputParser()

## Running the chain

Notice that in this case we are passing the data into the variable `s3_data` for use in the LLM.

In [ ]:
summary = chain.invoke({"s3_data": s3_data})
print("Summary:", summary)